In [11]:
import os
from ranger import Ranger

import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import Normalize
from torchmetrics import Accuracy

import torch.optim as optim
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc

import warnings
warnings.filterwarnings('ignore')

In [12]:
print(torch.cuda.is_available())

True


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
batch_size = 64
epochs = 100
train_path = '../datasets/imagenette2/train'
val_path = '../datasets/imagenette2/val'


In [15]:
train_dataloader = DataLoader(datasets.ImageFolder(train_path, transform = transforms.Compose([
                                                                    transforms.RandomResizedCrop(224),
                                                                    transforms.RandomHorizontalFlip(),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std=[0.229, 0.224, 0.225])
                                                            ])), batch_size = batch_size, shuffle=True)

test_dataloader = DataLoader(datasets.ImageFolder(val_path,
                                                               transform=transforms.Compose([
                                                                   transforms.ToTensor(),
                                                                   transforms.Resize([224, 224]),
                                                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std=[0.229, 0.224, 0.225])
                                                               ])),batch_size=batch_size, shuffle=True)

In [16]:
classes = ('tench', 'springer', 'casette_player', 'chain_saw','church', 'French_horn', 'garbage_truck', 'gas_pump', 'golf_ball', 'parachute')


In [17]:
%run models.ipynb
%run utils.ipynb

In [18]:
model_normal = resnet18_features(pretrained=False, filter='None', filter_layer=0)
learning_rate = 1e-4
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model_normal.parameters(), lr = learning_rate, weight_decay=1e-2, eps = 1e-06)
print(device)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
cuda


In [19]:
def train_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [20]:
model_normal = train_model(model = model_normal.to(device), epochs = epochs)

Epoch 1/100 - test accuracy: 31.97% and CE loss 1.86
Epoch 2/100 - test accuracy: 48.36% and CE loss 1.46
Epoch 3/100 - test accuracy: 55.90% and CE loss 1.25
Epoch 4/100 - test accuracy: 59.41% and CE loss 1.27
Epoch 5/100 - test accuracy: 62.24% and CE loss 1.18
Epoch 6/100 - test accuracy: 65.68% and CE loss 1.02
Epoch 7/100 - test accuracy: 64.94% and CE loss 0.92
Epoch 8/100 - test accuracy: 67.16% and CE loss 0.96
Epoch 9/100 - test accuracy: 69.35% and CE loss 1.16
Epoch 10/100 - test accuracy: 69.20% and CE loss 1.16
Epoch 11/100 - test accuracy: 70.17% and CE loss 0.82
Epoch 12/100 - test accuracy: 71.64% and CE loss 1.07
Epoch 13/100 - test accuracy: 71.85% and CE loss 0.84
Epoch 14/100 - test accuracy: 71.67% and CE loss 0.90
Epoch 15/100 - test accuracy: 74.14% and CE loss 0.96
Epoch 16/100 - test accuracy: 73.12% and CE loss 0.74
Epoch 17/100 - test accuracy: 74.32% and CE loss 1.05
Epoch 18/100 - test accuracy: 75.64% and CE loss 0.69
Epoch 19/100 - test accuracy: 74.85% 

In [21]:
# Model to GPU and eval mode.
model_normal.to(device)
model_normal.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 82.78%


In [22]:
from pathlib import Path

MODEL_PATH = Path("../models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "resnet_imagenette.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_normal.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: ..\models\resnet_imagenette.pth
